# Start Interactive Dask Cluster

In [1]:
pip install --upgrade git+https://github.com/FredFLi/dask-cloudprovider@fredfli/compute

g upgrade: attrs>=17.3.0 in c:\users\copeters\appdata\local\continuum\miniconda3\envs\simple\lib\site-packages (from aiohttp>=3.3.1->aiobotocore>=0.10.2->dask-cloudprovider==0.1.1+283.ga6550e0) (19.1.0)
  Stored in directory: C:\Users\copeters\AppData\Local\Temp\pip-ephem-wheel-cache-f75pnkj6\wheels\d5\f7\c2\0fd5b14cea01af1232aa8b7676b4caeaa600af85bf56c67f63
Successfully built dask-cloudprovider
  Found existing installation: dask-cloudprovider 0.1.1+282.gd156426
    Uninstalling dask-cloudprovider-0.1.1+282.gd156426:
      Successfully uninstalled dask-cloudprovider-0.1.1+282.gd156426
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade azureml-sdk

Requirement already up-to-date: azureml-sdk in c:\users\copeters\appdata\local\continuum\miniconda3\envs\simple\lib\site-packages (1.8.0)
Note: you may need to restart the kernel to use updated packages.


**RESTART YOUR KERNEL**

## Workspace

Get the Azure ML workspace.

In [1]:
from azureml.core import Workspace

In [2]:
ws = Workspace.from_config()
ws

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Workspace.create(name='AzureML', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='cody-eastus-rg')

### Start cluster

In [3]:
from dask_cloudprovider import AzureMLCluster

In [4]:
cluster = AzureMLCluster(ws)

................................................................




In [5]:
cluster.run

Experiment,Id,Type,Status,Details Page,Docs Page
dask-cloudprovider,dask-cloudprovider_1593783788_166b26d2,azureml.scriptrun,Running,Link to Azure Machine Learning studio,Link to Documentation


In [6]:
cluster.scale(30) # need more than default quota for this 

In [7]:
cluster

In [8]:
#cluster.close()

In [9]:
help(AzureMLCluster)

Help on class AzureMLCluster in module dask_cloudprovider.providers.azure.azureml:

class AzureMLCluster(distributed.deploy.cluster.Cluster)
 |  AzureMLCluster(workspace, compute_target=None, environment_definition=None, experiment_name=None, initial_node_count=None, jupyter=None, jupyter_port=None, dashboard_port=None, scheduler_port=None, scheduler_idle_timeout=None, worker_death_timeout=None, additional_ports=None, admin_username=None, admin_ssh_key=None, datastores=None, code_store=None, vnet=None, subnet=None, debug_mode=False, telemetry_opt_out=None, asynchronous=False, **kwargs)
 |  
 |  Deploy a Dask cluster using Azure ML
 |  
 |  This creates a dask scheduler and workers on an Azure ML Compute Target.
 |  
 |  Parameters
 |  ----------
 |  workspace: azureml.core.Workspace (required)
 |      Azure ML Workspace - see https://aka.ms/azureml/workspace
 |  
 |  compute_target: azureml.core.ComputeTarget (required)
 |      Azure ML Compute Target - see https://aka.ms/azureml/compu